In [26]:
pip install tensorflow opencv-python numpy pandas scikit-learn matplotlib seaborn


Note: you may need to restart the kernel to use updated packages.


In [32]:
import cv2
import os
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Función para extraer frames de un vídeo a intervalos dados por sample_rate
def extract_frames(video_path, sample_rate=30, resize_dim=(160, 160)):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # Frames por segundo del vídeo
    frames = []
    
    # Se toma un frame cada 'sample_rate' frames
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        if count % sample_rate == 0:
            frame = cv2.resize(frame, resize_dim)  # Redimensionar frame a 160x160
            frames.append(frame)
        
        count += 1
    
    cap.release()
    return np.array(frames)

# Función para cargar y preprocesar vídeos de un conjunto reducido (5 deportes)
def load_videos_from_dataset(dataset_path, sample_rate=30, max_len=20, max_videos_per_class=10):
    video_paths = []
    labels = []
    
    selected_classes = ['Yoyo', 'Basketball', 'Pushups', 'Nunchucks', 'Tennis']
    
    for class_idx, class_name in enumerate(selected_classes):
        class_folder = os.path.join(dataset_path, class_name)
        if os.path.isdir(class_folder):
            videos = os.listdir(class_folder)[:max_videos_per_class]  # Limitar a 10 vídeos por clase
            for video_name in videos:
                video_path = os.path.join(class_folder, video_name)
                video_paths.append(video_path)
                labels.append(class_idx)
    
    X = []
    for video_path in video_paths:
        frames = extract_frames(video_path, sample_rate)
        if len(frames) > max_len:
            frames = frames[:max_len]
        else:
            frames = np.pad(frames, ((0, max_len - len(frames)), (0, 0), (0, 0), (0, 0)), 'constant')
        X.append(frames)
    
    return np.array(X), np.array(labels)

# Construcción de la CNN (Red Neuronal Convolucional)
def build_cnn(input_shape=(160, 160, 3)):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5)  # Añadir Dropout para regularización
    ])
    return model

# Modelo CNN + LSTM
def build_model(input_shape=(20, 160, 160, 3), num_classes=5):
    cnn_model = build_cnn(input_shape[1:])
    model = models.Sequential([
        layers.TimeDistributed(cnn_model, input_shape=input_shape),
        layers.LSTM(128, return_sequences=False),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),  # Dropout adicional
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Cargar el dataset reducido con 5 deportes (Yoyo, Basketball, Pushups, Nunchucks, Tennis)
dataset_path = '/Users/mauro/Documents/MIA/EDEM_MDA2425/ALUMNOS/MIA/MAURO_BALAGUER/RN3_RNN/UCF50'  # Asegúrate de que este path sea correcto
X_train, Y_train = load_videos_from_dataset(dataset_path, sample_rate=30, max_len=20, max_videos_per_class=10)

# Convertir las etiquetas a formato one-hot
Y_train_one_hot = to_categorical(Y_train, num_classes=5)

# Aseguramos que las secuencias de entrada tengan la forma correcta
X_train_padded = np.array(X_train)

# Construir el modelo CNN + LSTM
model = build_model()  # Usamos la función build_model definida previamente

# Entrenamiento del modelo utilizando fit() que tiene soporte nativo para barras de progreso
model.fit(X_train_padded, Y_train_one_hot, epochs=10, batch_size=2, validation_split=0.2)

# Guardar el modelo entrenado
model.save('sports_classifier_5_sports.h5')

# Evaluación final en el conjunto de entrenamiento
loss, accuracy = model.evaluate(X_train_padded, Y_train_one_hot)
print(f"Accuracy en el conjunto de entrenamiento: {accuracy * 100:.2f}%")


Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 401ms/step - accuracy: 0.1597 - loss: 1.5780 - val_accuracy: 0.0000e+00 - val_loss: 2.7498
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 378ms/step - accuracy: 0.2667 - loss: 1.5203 - val_accuracy: 0.0000e+00 - val_loss: 2.2504
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 367ms/step - accuracy: 0.3579 - loss: 1.3590 - val_accuracy: 0.0000e+00 - val_loss: 2.9884
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 375ms/step - accuracy: 0.2491 - loss: 1.3350 - val_accuracy: 0.0000e+00 - val_loss: 2.8369
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 372ms/step - accuracy: 0.2504 - loss: 1.3602 - val_accuracy: 0.0000e+00 - val_loss: 2.8374
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 382ms/step - accuracy: 0.2886 - loss: 1.4728 - val_accuracy: 0.0000e+00 - val_loss: 2.7042
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 392ms/step - accuracy: 0.0649 - loss: 1.2907 - val_accuracy: 0.0000e+00 - val_loss: 3.2316
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 398ms/step - accuracy: 0.3192 - los

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 352ms/step - accuracy: 0.2708 - loss: 1.5195
Accuracy en el conjunto de entrenamiento: 25.00%


El modelo **CNN + LSTM** implementado logra un **25% de accuracy en entrenamiento**, lo cual indica un rendimiento bajo y cercano al azar (20% en un problema de 5 clases). Esto se debe principalmente a:

- Pocos datos: solo 10 vídeos por clase no son suficientes para entrenar un modelo profundo.
- Modelo demasiado complejo para el tamaño del dataset.
- Sampling limitado (1 frame cada 30), que puede omitir información relevante.

**🔧 Posibles mejoras:**

- Aumentar el número de vídeos y variedad de datos.
- Usar una CNN preentrenada para extraer mejores características.
- Capturar más frames por vídeo.
- Entrenar con más **potencia computacional (GPUs)** para probar modelos más avanzados.